- https://github.com/ksopyla/awesome-nlp-polish
- Sentiment: https://pypi.org/project/sentimentpl/
- Auto correct: https://github.com/filyp/autocorrect
- other: https://github.com/sdadas/polish-nlp-resources
- papers: https://homados.ipipan.waw.pl/?page_id=93

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=7,progress_bar=True)

from parallelbar import progress_map

# import spacy
# from sentimentpl.models import SentimentPLModel
# from autocorrect import Speller

# nlp_core = spacy.load("pl_core_news_lg")
# model = SentimentPLModel(from_pretrained='latest')
# spell = Speller('pl')

from utils import *

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# CZ data

In [2]:
df_pl_cz = pd.read_csv('../datasets/demagog_nlp_cz/converted-exp-PL.tsv', sep='\t')

df_pl_cz['text_clean'] = df_pl_cz['statementText'].apply(lambda x: clean_przyp(x))

tasks = df_pl_cz['text_clean'].values.tolist()
result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)

Core 1:   0%|          | 0/405 [00:00<?, ?it/s]

Core 2:   0%|          | 0/405 [00:00<?, ?it/s]

Core 3:   0%|          | 0/405 [00:00<?, ?it/s]

Core 4:   0%|          | 0/405 [00:00<?, ?it/s]

Core 5:   0%|          | 0/405 [00:00<?, ?it/s]

Core 6:   0%|          | 0/405 [00:00<?, ?it/s]

Core 7:   0%|          | 0/405 [00:00<?, ?it/s]

In [4]:
df_pl_cz['raw_dict'] = result 

df_clean = (
    df_pl_cz.join( 
        df_pl_cz['raw_dict']
        .apply(pd.Series)
    ).drop('raw_dict', axis=1)
)

df_clean['TEXT_POS'] = df_clean['TEXT_POS'].str.join(" ")

In [5]:
df_clean.to_csv('../datasets/ready2use/fake_news_features_cz.csv', sep=';', index=False, encoding='utf8')

# Demagog

In [6]:
df = pd.read_csv('../datasets/scrapped/demagog.csv', sep=';')

df = df.dropna()
df = df[df['text'].str.len() > 0 ]

df['text_clean'] = df['text'].apply(lambda x: clean_przyp(x))

df = df[df['text_clean'].str.len() > 1 ]

tasks = df['text_clean'].values.tolist()
result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)

Core 1:   0%|          | 0/703 [00:00<?, ?it/s]

Core 2:   0%|          | 0/703 [00:00<?, ?it/s]

Core 3:   0%|          | 0/703 [00:00<?, ?it/s]

Core 4:   0%|          | 0/703 [00:00<?, ?it/s]

Core 5:   0%|          | 0/703 [00:00<?, ?it/s]

Core 6:   0%|          | 0/703 [00:00<?, ?it/s]

Core 7:   0%|          | 0/703 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
df['raw_dict'] = result 

df_clean = (
    df.join( 
        df['raw_dict']
        .apply(pd.Series)
#         .rename(columns={'sentiment_lemm' : 'uniq_lemm'}) 
    ).drop('raw_dict', axis=1)
)

df_clean['TEXT_POS'] = df_clean['TEXT_POS'].str.join(" ")

In [ ]:
df_clean.to_csv('../datasets/scrapped/demagog_features.csv', sep=';', index=False)

# OKO.press

In [ ]:
df_oko_raw = pd.read_csv('../datasets/oko.press/query_result.tsv', sep='\t')

print(df_oko_raw['Id wypowiedzi'].unique().shape)

df_oko = pd.merge(
    df_oko_raw.pivot(index=['Id wypowiedzi'], columns='Nazwa pola danych', values='Wartość pola danych').reset_index(),
    df_oko_raw[~df_oko_raw['Autor Wypowiedzi'].isin(['Link do hasła', 'Nazwa słupka', 'Wesprzyj nas'])] \
        [['Id wypowiedzi', 'Autor Wypowiedzi']].dropna(),
    on='Id wypowiedzi',
    how='left'
)

print(df_oko.shape)

In [ ]:
df_oko_fin = df_oko[['sub_napis', 'sub_stan_zegara', 'Autor Wypowiedzi']]

df_oko_fin['text_clean'] = df_oko_fin['sub_napis'].progress_apply(lambda x: clean_przyp(x))

tasks = df_oko_fin['text_clean'].values.tolist()
result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)

In [ ]:
df_oko_fin['raw_dict'] = result

df_oko_clean = df_oko_fin.join( df_oko_fin['raw_dict'].apply(pd.Series) ).drop('raw_dict', axis=1)

df_oko_clean['TEXT_POS'] = df_oko_clean['TEXT_POS'].str.join(" ")

In [ ]:
df_oko_clean.to_csv('../datasets/oko.press/okopress_features.csv', sep=';', index=False)